# SQL + Python integration

Python allows us to connect and manage SQLite database. In order to do that we use module called sqlite3.
The sqlite3 module was written by Gerhard Häring. It provides a SQL interface compliant with the DB-API 2.0.


The DB API provides a minimal standard for working with databases, using Python structures and syntax wherever possible. This API includes the following:

    Connections, which cover guidelines for how to connect to databases

    Executing statements and stored procedures to query, update, insert, and delete data with cursors

    Transactions, with support for committing or rolling back a transaction

    Examining metadata on the database module as well as on database and table structure

    Defining the types of errors


# Create\Close Connection

To use sqlite3 module we have to first import it and than we create a Connection object (cnn) that represents the database.
We pass the database file name (should be located in the same directory as jupyter notebook file) as parameter in connect method. 

In [ ]:
import sqlite3
# We connect to testdb2.db database
conn = sqlite3.connect('testdb2.db')


# We close the connection and free all resources
conn.close()

# Create table

Once we have object Connection (conn), we can create an Cursor object (c). The c object allows us to create table students by callling the method execute. We pass creat table command as parameter in execute method.



In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# Create table
c.execute("CREATE TABLE students(student_id integer primary key autoincrement,name text not null, surname text not null,birth date,weight int,height int)")

# Save (commit) the changes in database.
# Changes not commited will be lost
conn.commit()

# We close the connection and free all resources
conn.close()


We ca n ru a query to what tables there are in our database.

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
for tableName in tables:
    print(tableName)

# We do not have to commit anything as we didn't make any changes.
conn.close()

We use the method commit() of Cursor object to save changes in database.
Finally, we close the connection with close() method to free resources.

# DROP table

We use method execute and sql drop statment to remove table from database.

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# execute the commend below to drop table

c.execute("Drop table students")
# Save (commit) the changes
conn.commit()

# We close the connection and free all resources
conn.close()

# Insert rows

We put data into a table students, using sql insert statment in execute command.

The values in sql insert command should be passed with placeholder operator (?).

The use of string operators (not placeholder operator) is a bad programming practice.
It results in the thread of sql injection attack. Humorous description of that problem we find at  https://xkcd.com/327/ .


First create table:

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# Create table
c.execute("CREATE TABLE students(student_id integer primary key autoincrement,name text not null, surname text not null,birth date,weight int,height int)")

# Save (commit) the changes in database.
# Changes not commited will be lost
conn.commit()

# We close the connection and free all resources
conn.close()

Now,let's insert records:

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# Insert record values using placeholder operator
# While student_id column value is autoincrement primary key,its value should be equal to None
c.execute("INSERT INTO students VALUES(?,?,?,?,?,?)",(None,'Tom','Silver',72,182,'1989-11-03'))

# WRONG:Do not use string operators as below
#c.execute("INSERT INTO students VALUES ({0},{1},{2},{3},{4})".format('Tom','Silver',72,182,'1989-11-03'))

# print total number of changed rows
print("number of affected rows: {0}".format(conn.total_changes))

# Save (commit) the changes
conn.commit()

# We close the connection and free all resources
conn.close()

We can also insert many records in one time into database. We use for that executemany() method. We pass list of records as argument. Each record is also a single list. 

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# Larger example that inserts many records at a time
studentsRecords = [(None,'Tom','Silver',72,182,'1989-11-03'),
             (None,'Adam','Brown',82,192,'1992-11-03'),
             (None,'Maria','Great',52,162,'1995-11-03'),]

c.executemany('INSERT INTO students VALUES (?,?,?,?,?,?)', studentsRecords)

# print total number of changed rows
print("number of affected rows: {0}".format(conn.total_changes))

# Save (commit) the changes
conn.commit()

# We close the connection and free all resources
conn.close()

# Select rows

To retrieve data after executing a SELECT statement, you can either treat the cursor as an iterator, call the cursor’s fetchone() method to retrieve a single matching row, or call fetchall() to get a list of the matching rows

In [ ]:
conn = sqlite3.connect('testdb2.db')

c = conn.cursor()

# execute the commend below to select name,surname and iteterate through results using iterator
for row in c.execute("SELECT name,surname FROM students ORDER BY surname"):
    print(row)

c.execute("SELECT name,surname FROM students ORDER BY surname")

# execute the commend below to select name,surname,weight,height
c.execute("SELECT name,surname,weight,height FROM students ORDER BY surname")

# get and print single result
print(c.fetchone())


# execute the commend below to select name,surname,weight,height
c.execute("SELECT * from students ORDER BY surname")

# get all results,assign them to the list,fecthall() returns empty list if no results
listOfResults=c.fetchall()
for item in listOfResults:
    print(item)


# Save (commit) the changes
conn.commit()

# We close the connection and free all resources
conn.close()

If we want to obtain information about number of columns and columns names then we shoud operate on Row object. Row object is returned by fetchone() method. 

REMARK: We must define row_factory in Connection object as below.

In [ ]:
conn = sqlite3.connect('testdb2.db')

# If we want to operate on rows we must define row factory
conn.row_factory = sqlite3.Row

c = conn.cursor()

c.execute("SELECT * from students ORDER BY surname")
row=c.fetchone()

# print number of columns
print(len(row))
# print value in the first three column 
print(row[0], row[1], row[2])
# or address values by column names.
print(row["name"], row["surname"])
# print columns names
print(row.keys())

# We close the connection and free all resources
conn.close()


# Custom function

SQLite3 module allows user to define custom function. Below we have example of md5sum(t) function, which encodes the input string. 

In [ ]:
# import required modules
import hashlib

def md5sum(t):
    return hashlib.md5(t).hexdigest()

conn = sqlite3.connect("testdb2.db")
# create function takes three arguments: 
# name of the custom function in SQLite3 module, number of parameters,name of the custom function in Python 
conn.create_function("md5", 1, md5sum)
c = conn.cursor()
# below code encodes name,surname and inserts data into table
c.execute("INSERT INTO students VALUES (?,md5(?),?,?,?,?)",(None,b'Tom','Silver',72,182,'1989-11-03'))

# print total number of changed rows
print("number of affected rows: {0}".format(conn.total_changes))

c.execute("Select * from students")
# get all results,assign them to the list,fecthall() returns empty list if no results
listOfResults=c.fetchall()
for item in listOfResults:
    print(item)

# Save (commit) the changes
conn.commit()

# We close the connection and free all resources
conn.close()

# Controlling Transactions


The sqlite3 module opens transactions implicitly before a SQL statement (i.e. INSERT/UPDATE/DELETE/REPLACE). 
The method rollback() method rolls back any changes to the database since the last call to commit().

# Exceptions

SQLite3 module provides the following types of exceptions:

*exception sqlite3.DatabaseError

    Exception raised for errors that are related to the database.

*exception sqlite3.IntegrityError

    Exception raised when the relational integrity of the database is affected, e.g. a foreign key check fails. 
    It is a subclass of DatabaseError.

*exception sqlite3.ProgrammingError

    Exception raised for programming errors, e.g. table not found or already exists, 
    syntax error in the SQL statement, wrong number of parameters specified, etc. 
    It is a subclass of DatabaseError.

REMARK: We should remember to close connection and free resources in both cases when exception occurs and does not occur.
When exception occurs we should rollback changes in given transaction.


In [ ]:
con = sqlite3.connect("testdb2.db")

try:
    # Successful, con.commit() is called automatically afterwards
    with con:
        con.execute("INSERT INTO students VALUES (?,?,?,?,?,?)",(None,'Mark','LastGood',69,174,'1989-11-03'))
        con.execute("INSERT INTO students VALUES (?,?,?,?,?,?)",(None,'Mark',None,69,174,'1989-11-03'))
        con.execute("INSERT INTO students VALUES (?,?,?,?,?,?)",('Tom',None,72,182,'1989-11-03'))

        # When exception occurs, we use rollback() method to revert changes 
except sqlite3.IntegrityError:
    print("IntegrityError:couldn't add record with null surname")
    con.rollback()
except sqlite3.ProgrammingError:
    print("ProgrammingError:table does not exist")
    con.rollback()
except sqlite3.Error:
    print("Error:general db erorr")
    con.rollback()
    
# We close the connection and free all resources
conn.close()